In [1]:
import os
import re
import pymatgen as mg
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import glob
import matplotlib.pyplot as plt

In [2]:
import paddle
from paddle import fluid
#from paddle_quantum.circuit import UAnsatz
from paddle_quantum.ansatz import Circuit
from paddle_quantum.qinfo import state_fidelity, partial_trace, pauli_str_to_matrix, NKron
from paddle_quantum.linalg import dagger, haar_unitary
#from paddle_quantum.utils import partial_trace, dagger, state_fidelity, NKron,pauli_str_to_matrix
from paddle import complex
from paddle import matmul, trace, kron#,elementwise_add

In [3]:
import numpy as np
from sklearn import preprocessing

In [4]:
import xrd4QML
from xrd4QML import tomgStructure, get_xrdmat

In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [6]:
import paddle_qcnn_5bits_5to4

/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


In [7]:
torch.set_default_dtype(torch.float32)
paddle.set_default_dtype("float32")

In [8]:
dir_file = "/Users/junjiehu/Mlearning/MaterialsML/DFT-bluk-cpx222/"

In [9]:
def xrdpre_process(xrd_path):
    xrd1=xrd_path
    xrd2=tomgStructure(xrd1)
    xrd3=np.nan_to_num(get_xrdmat(xrd2))
    xrd4=preprocessing.normalize(np.array([xrd3]), norm="l2", axis=1)
    xrd5=np.dot(xrd4.T,xrd4)
    xrd6=xrd5.astype('complex64')
    return xrd6

In [10]:
theta_load=np.asarray([1.50826, 2.16304, 3.54938, 3.59472, 2.57092, 5.31218, 3.19765, -0.364409,
                       5.19505, 4.66239, 6.03654, 2.37409, 2.89946, 0.0742845, 0.731149, 5.05812, 
                       2.37495, -0.540096, 3.20501, 3.07696, -1.39899])

In [11]:
class Net_3(nn.Module):
    def __init__(self):
        super(Net_3, self).__init__()
        
        self.out=nn.Sequential(
            nn.Linear(4,32),
            nn.ReLU(),
            nn.Linear(32,1),            
        )
        
    def forward(self,x):
        x=x.view(x.size(0),-1)
        output=self.out(x)
        return output

In [12]:
workfile=[]
for m1,n1,fname in os.walk(dir_file):
    for ieach in n1:
        ieach=dir_file+ieach
        workfile.append(ieach)

In [13]:
loss_set=[]
load_net1=paddle_qcnn_5bits_5to4.load_NET_2(theta_load)
HQMLmodel= Net_3()
HQMLmodel.load_state_dict(torch.load("/Users/junjiehu/Desktop/202210test/net3.pth"))

<All keys matched successfully>

In [14]:
DFT_energy=[]
QML_energy=[]
for path_ in workfile:
    try:
        dft_energy= xrd4QML.get_energy(path_)
        input_xrd=xrdpre_process(path_)
        a_out,c_encode=load_net1(input_xrd)
        input_net3=paddle_qcnn_5bits_5to4.measure_Z(c_encode)
        input_net3=np.asarray([[input_net3]])
        input_net3=np.asarray(input_net3,dtype=np.float64)
        input_net3=torch.Tensor(input_net3)
        Gout=HQMLmodel(input_net3)
        G_data=Gout.data.numpy().mean()
        QML_energy.append(xrd4QML.inverse_transform(G_data))
        DFT_energy.append(dft_energy)
    except:
        print(path_)

/Users/junjiehu/Desktop/202210test/cpx/5to4/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/Users/junjiehu/Desktop/202210test/cpx/5to4/paddle_qcnn_5bits_5to4.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  x = x.astype('float32')
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this war

/Users/junjiehu/Desktop/202210test/cpx/5to4/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/Users/junjiehu/Desktop/202210test/cpx/5to4/paddle_qcnn_5bits_5to4.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  x = x.astype('float32')
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this war

/Users/junjiehu/Desktop/202210test/cpx/5to4/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/Users/junjiehu/Desktop/202210test/cpx/5to4/paddle_qcnn_5bits_5to4.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  x = x.astype('float32')
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this war

/Users/junjiehu/Desktop/202210test/cpx/5to4/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/Users/junjiehu/Desktop/202210test/cpx/5to4/paddle_qcnn_5bits_5to4.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  x = x.astype('float32')
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this war

/Users/junjiehu/Desktop/202210test/cpx/5to4/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/Users/junjiehu/Desktop/202210test/cpx/5to4/paddle_qcnn_5bits_5to4.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  x = x.astype('float32')
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this war

...

/Users/junjiehu/Desktop/202210test/cpx/5to4/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/Users/junjiehu/Desktop/202210test/cpx/5to4/paddle_qcnn_5bits_5to4.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  x = x.astype('float32')
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this war

/Users/junjiehu/Desktop/202210test/cpx/5to4/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/Users/junjiehu/Desktop/202210test/cpx/5to4/paddle_qcnn_5bits_5to4.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  x = x.astype('float32')
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this war

/Users/junjiehu/Desktop/202210test/cpx/5to4/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/Users/junjiehu/Desktop/202210test/cpx/5to4/paddle_qcnn_5bits_5to4.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  x = x.astype('float32')
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this war

/Users/junjiehu/Desktop/202210test/cpx/5to4/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/Users/junjiehu/Desktop/202210test/cpx/5to4/paddle_qcnn_5bits_5to4.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  x = x.astype('float32')
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this war

/Users/junjiehu/Desktop/202210test/cpx/5to4/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/Users/junjiehu/Desktop/202210test/cpx/5to4/paddle_qcnn_5bits_5to4.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  x = x.astype('float32')
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/Users/junjiehu/opt/anaconda3/envs/ML/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this war

In [18]:
np.min(QML_energy)

-122.6941094586277

In [20]:
np.min(DFT_energy)

-122.70765

In [26]:
np.mean(np.abs(np.asarray(QML_energy)-np.asarray(DFT_energy)))/40

0.00019061588108717692